# EEG Signals Classification

## About Dataset 


Acquisition Data:
For the capture of brain impulses, an EMOTIV EPOC + 14 Channel electroencephalogram from EMOTIV was used. This equipment has 128 Hz sampling frequency with 16-bit analog analog converter with 14 electrode channels: AF3, F7, F3, FC5, T7, P7, O1, O2, P8, T8, FC6, F4, F8 and AF4.

For the purpose of controlling the virtual object, a twenty five minute data collection protocol was created with different cycles that address different situations while the user tries to control the object. Different situations have been proposed as part of the protocol so that the machine learning component can generalize brainwave behavior with respect to object control commands regardless of the situation.

During each of the acquisition cycles, the participant was exposed to the images depicting voluntary motor actions, namely: a right arrow that would represent motor action in the right direction, a left arrow that would represent motor action from the left direction and a circle that would represent no motor action. ̃

The only exception is the situation in which the participant is with his eyes closed, where a beep is given to the participant to open the eye only at specific times to view the images.

In [4]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import tensorflow as tf

In [20]:
dataset = [pd.read_csv('./Dataset/user_'+user +'.csv') for user in ['a','b','c','d']]

In [21]:
dataset[0]

,Class,AF3 delta std,AF3 delta m,AF3 theta std,AF3 theta m,AF3 alpha std,AF3 alpha m,AF3 beta std,AF3 beta m,F7 delta std,...,F8 beta std,F8 beta m,AF4 delta std,AF4 delta m,AF4 theta std,AF4 theta m,AF4 alpha std,AF4 alpha m,AF4 beta std,AF4 beta m
0,1.0,3569.164550,2063.892754,1.673726,4.444736,0.526209,3.002088,1.425022,3.302739,3563.803888,...,45.468326,72.508750,3701.186330,2182.676835,18.192418,41.349662,16.004756,42.046467,46.280843,73.565719
1,1.0,3568.423670,2063.099248,1.897790,3.728823,1.304186,1.854353,1.366575,2.546458,3563.560922,...,36.551948,66.931186,3725.210509,2180.197439,8.820788,38.012788,19.601233,29.431054,38.559351,67.470041
2,1.0,3568.157929,2062.445859,2.798014,2.574504,1.120537,1.958819,0.982433,2.258622,3563.279981,...,40.754308,66.816547,3724.417296,2176.823208,18.159202,23.612639,14.378291,19.555084,43.210004,67.781924
3,1.0,3567.710021,2062.112673,2.181775,3.610507,0.629608,2.155876,0.856275,2.233711,3562.787801,...,38.074628,63.915386,3725.822160,2177.089059,19.737616,29.484396,15.793034,25.713513,39.250246,65.031031
4,1.0,3565.546124,2063.128867,1.685161,3.384311,0.677526,1.795798,0.927924,1.909810,3562.655091,...,35.357384,64.534645,3723.053978,2167.798335,8.429414,26.374975,14.920736,35.675266,33.901687,66.956313
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2875,0.0,3572.820630,2064.823341,0.676923,1.620879,0.965369,2.392263,2.582266,3.185588,3565.166724,...,26.247669,13.320391,3607.467804,2089.263161,1.605773,4.247634,1.649762,2.938868,31.019482,16.648758
2876,0.0,3573.915291,2065.845542,0.860279,1.504333,0.539180,1.331165,2.533695,2.870270,3565.734750,...,26.120266,14.809811,3613.165565,2089.298802,0.643899,6.903858,1.287563,3.942647,31.546882,17.429113
2877,0.0,3574.326202,2066.031013,1.416731,2.581577,0.715012,1.307975,2.261013,2.873613,3567.582880,...,26.513829,16.084048,3615.755534,2094.482958,1.851314,5.683580,2.411642,6.355117,32.238063,19.023897
2878,0.0,3573.735788,2065.945333,1.119943,2.769816,0.455408,1.541430,1.782814,3.067384,3567.857997,...,20.049092,21.248860,3620.821019,2097.450420,5.035767,12.027902,6.441267,14.663681,24.109509,26.468254


#### Adding User column

In [22]:
dfs = dataset.copy()
for i in range(len(dfs)):
    dfs[i]['User'] = pd.Series(i, index=dfs[i].index)

In [10]:
dfs

[      Class  AF3 delta std  AF3 delta m  AF3 theta std  AF3 theta m  \
 0       1.0    3569.164550  2063.892754       1.673726     4.444736   
 1       1.0    3568.423670  2063.099248       1.897790     3.728823   
 2       1.0    3568.157929  2062.445859       2.798014     2.574504   
 3       1.0    3567.710021  2062.112673       2.181775     3.610507   
 4       1.0    3565.546124  2063.128867       1.685161     3.384311   
 ...     ...            ...          ...            ...          ...   
 2875    0.0    3572.820630  2064.823341       0.676923     1.620879   
 2876    0.0    3573.915291  2065.845542       0.860279     1.504333   
 2877    0.0    3574.326202  2066.031013       1.416731     2.581577   
 2878    0.0    3573.735788  2065.945333       1.119943     2.769816   
 2879    0.0    3573.982930  2065.960401       1.011062     2.696217   
 
       AF3 alpha std  AF3 alpha m  AF3 beta std  AF3 beta m  F7 delta std  ...  \
 0          0.526209     3.002088      1.425022    3

#### Concatenating Datasets

In [24]:
data = pd.concat(dfs, axis=0).sample(frac=1.0, random_state=123).reset_index(drop=True)

In [25]:
data

,Class,AF3 delta std,AF3 delta m,AF3 theta std,AF3 theta m,AF3 alpha std,AF3 alpha m,AF3 beta std,AF3 beta m,F7 delta std,...,F8 beta m,AF4 delta std,AF4 delta m,AF4 theta std,AF4 theta m,AF4 alpha std,AF4 alpha m,AF4 beta std,AF4 beta m,User
0,0.0,3572.252440,2065.056469,0.851824,2.047953,0.651871,2.522036,2.109733,3.347705,3564.779879,...,35.151586,3628.426885,2129.789645,5.353671,17.885132,7.672209,29.960618,43.216980,43.932669,0
1,1.0,3574.116024,2065.528155,1.932513,3.321636,1.138012,2.349805,2.256212,3.945981,3563.399422,...,40.800889,3680.341349,2144.200503,10.819521,36.995982,12.812193,24.146774,23.747501,49.072017,0
2,0.0,3554.487593,2056.215665,0.935015,3.793783,0.736168,2.680542,3.381325,4.678876,3568.839949,...,18.176841,3538.347368,2081.315814,5.486555,13.204753,0.664075,6.633072,1.434277,4.132446,2
3,0.0,3570.668125,2063.974908,1.875394,3.028541,0.910000,3.018672,1.163312,2.883009,3565.403408,...,55.547547,3604.601528,2122.493834,15.611283,16.452483,22.462175,46.703612,32.213578,70.892466,0
4,0.0,3559.747108,2057.401763,1.053691,1.530594,1.593121,2.789907,2.668865,4.326693,3573.651774,...,7.208052,3513.244789,2030.461207,1.455450,3.030659,0.482971,3.200647,0.895170,2.099638,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11515,2.0,3570.748191,2063.201099,0.727594,1.404708,0.995674,2.569811,0.991322,1.987807,3562.594707,...,1.910036,3567.687654,2062.791757,1.748982,2.097780,1.242056,1.682180,1.277379,1.997654,3
11516,1.0,3566.600695,2062.436502,1.491797,3.013466,1.338308,2.769207,4.268401,3.877338,3576.187723,...,5.217772,3548.277991,2055.537892,1.787661,3.366967,0.810903,1.739812,2.202195,3.294145,2
11517,0.0,3574.186933,2064.477869,0.710708,1.534898,0.749206,2.038258,2.208018,2.157076,3566.398415,...,2.200203,3572.290285,2063.776063,1.268569,2.547969,0.887808,1.651011,1.533523,1.957866,1
11518,0.0,3574.343116,2065.996679,0.606401,2.835015,0.617079,1.209722,3.357912,3.098423,3566.521533,...,15.413625,3608.394990,2091.509508,1.479338,3.824176,3.281520,4.238889,33.441194,19.158094,0


In [45]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11520 entries, 0 to 11519
Columns: 114 entries, Class to User
dtypes: float64(113), int64(1)
memory usage: 10.0 MB


In [26]:
data.dtypes

Class            float64
AF3 delta std    float64
AF3 delta m      float64
AF3 theta std    float64
AF3 theta m      float64
                  ...   
AF4 alpha std    float64
AF4 alpha m      float64
AF4 beta std     float64
AF4 beta m       float64
User               int64
Length: 114, dtype: object

In [46]:
data.describe()

,Class,AF3 delta std,AF3 delta m,AF3 theta std,AF3 theta m,AF3 alpha std,AF3 alpha m,AF3 beta std,AF3 beta m,F7 delta std,...,F8 beta m,AF4 delta std,AF4 delta m,AF4 theta std,AF4 theta m,AF4 alpha std,AF4 alpha m,AF4 beta std,AF4 beta m,User
count,11520.000000,11520.000000,11520.000000,11520.000000,11520.000000,11520.000000,11520.000000,11520.000000,11520.000000,11520.000000,...,11520.000000,11520.000000,11520.000000,11520.000000,11520.000000,11520.000000,11520.000000,11520.000000,11520.000000,11520.000000
mean,1.000000,3571.588010,2064.802117,1.042080,2.461821,0.909898,2.201861,2.197638,3.082371,3568.526257,...,11.538165,3578.750414,2077.076972,2.535672,6.408382,2.531363,6.396447,10.238555,11.640444,1.500000
std,0.816532,8.332205,5.111574,0.616193,1.240949,0.462773,0.834416,0.895613,0.837786,4.461650,...,14.459010,31.429308,26.881689,3.667870,9.503169,3.979140,9.289700,17.005642,17.357938,1.118083
min,0.000000,3370.595863,1978.862421,0.030171,0.479354,0.066521,0.479033,0.526885,1.308723,3555.254510,...,0.934593,2705.784083,2010.355030,0.047412,0.499164,0.043295,0.490850,0.465208,1.350438,0.000000
25%,0.000000,3569.067460,2063.029951,0.678118,1.902200,0.594002,1.678946,1.456163,2.397392,3564.948753,...,2.029585,3567.472989,2062.537954,0.799992,2.056858,0.688798,1.835439,1.217960,2.182373,0.750000
50%,1.000000,3571.816872,2064.547292,0.961184,2.312287,0.845206,2.116816,2.112369,2.947443,3567.678533,...,3.687725,3571.077723,2064.741919,1.239481,2.755133,1.062308,2.509870,1.593670,2.540830,1.500000
75%,2.000000,3574.115918,2065.889854,1.285127,2.775746,1.148914,2.610754,2.779936,3.650024,3571.772812,...,16.398057,3587.656882,2087.591409,2.317750,6.816659,1.994364,5.440804,6.415053,8.982841,2.250000
max,2.000000,3725.451803,2189.586102,15.206517,30.192580,9.030337,22.057474,5.574868,7.237756,3588.176094,...,89.132433,3758.046521,3005.693915,95.606765,244.145529,94.448386,138.908194,88.024693,94.350650,3.000000


#### Data Preprocessing

One Hot Encoding is a common way of preprocessing categorical features for machine learning models. This type of encoding creates a new binary feature for each possible category and assigns a value of 1 to the feature of each sample that corresponds to its original category.

In [28]:
def onehot_encode(df, column):
    df = df.copy()
    dummies = pd.get_dummies(df[column], prefix=column)
    df = pd.concat([df, dummies], axis=1)
    df = df.drop(column, axis=1)
    return df

In [29]:
onehot_encode(data, column='User')

,Class,AF3 delta std,AF3 delta m,AF3 theta std,AF3 theta m,AF3 alpha std,AF3 alpha m,AF3 beta std,AF3 beta m,F7 delta std,...,AF4 theta std,AF4 theta m,AF4 alpha std,AF4 alpha m,AF4 beta std,AF4 beta m,User_0,User_1,User_2,User_3
0,0.0,3572.252440,2065.056469,0.851824,2.047953,0.651871,2.522036,2.109733,3.347705,3564.779879,...,5.353671,17.885132,7.672209,29.960618,43.216980,43.932669,1,0,0,0
1,1.0,3574.116024,2065.528155,1.932513,3.321636,1.138012,2.349805,2.256212,3.945981,3563.399422,...,10.819521,36.995982,12.812193,24.146774,23.747501,49.072017,1,0,0,0
2,0.0,3554.487593,2056.215665,0.935015,3.793783,0.736168,2.680542,3.381325,4.678876,3568.839949,...,5.486555,13.204753,0.664075,6.633072,1.434277,4.132446,0,0,1,0
3,0.0,3570.668125,2063.974908,1.875394,3.028541,0.910000,3.018672,1.163312,2.883009,3565.403408,...,15.611283,16.452483,22.462175,46.703612,32.213578,70.892466,1,0,0,0
4,0.0,3559.747108,2057.401763,1.053691,1.530594,1.593121,2.789907,2.668865,4.326693,3573.651774,...,1.455450,3.030659,0.482971,3.200647,0.895170,2.099638,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11515,2.0,3570.748191,2063.201099,0.727594,1.404708,0.995674,2.569811,0.991322,1.987807,3562.594707,...,1.748982,2.097780,1.242056,1.682180,1.277379,1.997654,0,0,0,1
11516,1.0,3566.600695,2062.436502,1.491797,3.013466,1.338308,2.769207,4.268401,3.877338,3576.187723,...,1.787661,3.366967,0.810903,1.739812,2.202195,3.294145,0,0,1,0
11517,0.0,3574.186933,2064.477869,0.710708,1.534898,0.749206,2.038258,2.208018,2.157076,3566.398415,...,1.268569,2.547969,0.887808,1.651011,1.533523,1.957866,0,1,0,0
11518,0.0,3574.343116,2065.996679,0.606401,2.835015,0.617079,1.209722,3.357912,3.098423,3566.521533,...,1.479338,3.824176,3.281520,4.238889,33.441194,19.158094,1,0,0,0


In [31]:
def preprocess_inputs(df, target='Class'):
    df = df.copy()
    
    # One-hot encode whichever target column is not being used
    targets = ['Class', 'User']
    targets.remove(target)
    df = onehot_encode(df, column=targets[0])
    
    # Split df into X and y
    y = df[target].copy()
    X = df.drop(target, axis=1)
    
    # Train-test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, random_state=123)
    
    # Scale X with a standard scaler
    scaler = StandardScaler()
    scaler.fit(X_train)
    
    X_train = pd.DataFrame(scaler.transform(X_train), columns=X.columns)
    X_test = pd.DataFrame(scaler.transform(X_test), columns=X.columns)
    
    return X_train, X_test, y_train, y_test

In [39]:
X_train, X_test, y_train, y_test = preprocess_inputs(data, target='Class')


In [40]:
X_train # 70% of the data and all the columns are scaled.

,AF3 delta std,AF3 delta m,AF3 theta std,AF3 theta m,AF3 alpha std,AF3 alpha m,AF3 beta std,AF3 beta m,F7 delta std,F7 delta m,...,AF4 theta std,AF4 theta m,AF4 alpha std,AF4 alpha m,AF4 beta std,AF4 beta m,User_0,User_1,User_2,User_3
0,-0.592492,-0.628348,-0.961159,0.055905,0.463966,0.243742,-1.048130,-0.492935,-1.039009,-0.724031,...,-0.436216,-0.408949,-0.407029,-0.490410,-0.536367,-0.539566,-0.574343,-0.578735,-0.575870,1.722792
1,-1.215542,-1.060438,-0.687275,0.911497,0.266495,-0.045726,-0.996263,-1.073110,-1.595935,-1.789505,...,2.577270,2.173543,3.159511,3.022871,0.657025,2.073117,1.741120,-0.578735,-0.575870,-0.580453
2,0.217034,0.061461,-0.302453,-0.276674,-0.609120,-0.737363,-1.144599,-1.598413,-1.092270,-1.223951,...,-0.400908,-0.385079,-0.267008,-0.348359,-0.541364,-0.569416,-0.574343,-0.578735,-0.575870,1.722792
3,2.636827,3.089887,0.715433,1.688836,1.115520,0.633049,-0.753087,-0.640998,0.643631,0.970393,...,0.141047,-0.008948,-0.427868,-0.301202,-0.505539,-0.522675,-0.574343,-0.578735,-0.575870,1.722792
4,-0.012351,0.039718,-0.098249,-0.213172,0.948769,-0.058219,-0.254097,-0.487547,0.332959,0.028684,...,-0.565201,-0.479292,-0.372068,-0.497185,-0.523806,-0.525149,-0.574343,1.727908,-0.575870,-0.580453
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8058,0.618517,1.150023,-0.957440,-0.735247,0.634060,1.144167,1.837545,2.507360,1.076999,1.533723,...,-0.278668,0.153400,-0.380571,-0.152910,-0.500346,-0.453114,-0.574343,-0.578735,1.736502,-0.580453
8059,-0.157839,-0.326705,-0.099050,0.460753,-0.022372,-0.179817,1.098608,0.432161,-0.522874,-0.840956,...,2.779410,1.612038,1.694883,4.498588,3.159293,2.567176,1.741120,-0.578735,-0.575870,-0.580453
8060,0.166508,0.010035,0.018053,-0.268545,-1.090114,0.455020,0.757639,1.987062,0.999285,0.692423,...,-0.430248,-0.416324,-0.096056,-0.372181,-0.508712,-0.493789,-0.574343,-0.578735,1.736502,-0.580453
8061,0.202509,-0.017317,-0.267921,-0.589426,0.478543,-0.449163,-0.196672,-0.763750,-0.557585,-0.719922,...,-0.238893,-0.510057,-0.441859,-0.499372,-0.511246,-0.538075,-0.574343,1.727908,-0.575870,-0.580453


In [41]:
X_train.mean()  # The mean of each column is very close to zero, and the variances of each column is very close to one.

AF3 delta std   -1.764393e-14
AF3 delta m      4.312198e-14
AF3 theta std    7.992594e-17
AF3 theta m      1.044956e-16
AF3 alpha std   -1.768260e-16
                     ...     
AF4 beta m       3.629602e-17
User_0           8.966603e-17
User_1           2.724404e-16
User_2           3.796762e-16
User_3           8.195520e-17
Length: 116, dtype: float64

In [42]:
y_train

8701     1.0
4335     1.0
11203    1.0
11448    0.0
374      2.0
        ... 
9785     1.0
7763     2.0
5218     1.0
1346     1.0
3582     2.0
Name: Class, Length: 8063, dtype: float64

Using a standard keras 2 hidden layer neural network.


In [23]:
def build_model(num_classes=3):
    
    inputs = tf.keras.Input(shape=(X_train.shape[1],))
    x = tf.keras.layers.Dense(128, activation='relu')(inputs)
    x = tf.keras.layers.Dense(128, activation='relu')(x)
    outputs = tf.keras.layers.Dense(num_classes, activation='softmax')(x)
    
    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    
    model.compile(
        optimizer='adam',
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    
    return model

### Predicting Hand Movement Class

In [24]:
X_train, X_test, y_train, y_test = preprocess_inputs(data, target='Class')

In [25]:
X_train

,AF3 delta std,AF3 delta m,AF3 theta std,AF3 theta m,AF3 alpha std,AF3 alpha m,AF3 beta std,AF3 beta m,F7 delta std,F7 delta m,...,AF4 theta std,AF4 theta m,AF4 alpha std,AF4 alpha m,AF4 beta std,AF4 beta m,User_0,User_1,User_2,User_3
0,-0.592492,-0.628348,-0.961159,0.055905,0.463966,0.243742,-1.048130,-0.492935,-1.039009,-0.724031,...,-0.436216,-0.408949,-0.407029,-0.490410,-0.536367,-0.539566,-0.574343,-0.578735,-0.575870,1.722792
1,-1.215542,-1.060438,-0.687275,0.911497,0.266495,-0.045726,-0.996263,-1.073110,-1.595935,-1.789505,...,2.577270,2.173543,3.159511,3.022871,0.657025,2.073117,1.741120,-0.578735,-0.575870,-0.580453
2,0.217034,0.061461,-0.302453,-0.276674,-0.609120,-0.737363,-1.144599,-1.598413,-1.092270,-1.223951,...,-0.400908,-0.385079,-0.267008,-0.348359,-0.541364,-0.569416,-0.574343,-0.578735,-0.575870,1.722792
3,2.636827,3.089887,0.715433,1.688836,1.115520,0.633049,-0.753087,-0.640998,0.643631,0.970393,...,0.141047,-0.008948,-0.427868,-0.301202,-0.505539,-0.522675,-0.574343,-0.578735,-0.575870,1.722792
4,-0.012351,0.039718,-0.098249,-0.213172,0.948769,-0.058219,-0.254097,-0.487547,0.332959,0.028684,...,-0.565201,-0.479292,-0.372068,-0.497185,-0.523806,-0.525149,-0.574343,1.727908,-0.575870,-0.580453
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8058,0.618517,1.150023,-0.957440,-0.735247,0.634060,1.144167,1.837545,2.507360,1.076999,1.533723,...,-0.278668,0.153400,-0.380571,-0.152910,-0.500346,-0.453114,-0.574343,-0.578735,1.736502,-0.580453
8059,-0.157839,-0.326705,-0.099050,0.460753,-0.022372,-0.179817,1.098608,0.432161,-0.522874,-0.840956,...,2.779410,1.612038,1.694883,4.498588,3.159293,2.567176,1.741120,-0.578735,-0.575870,-0.580453
8060,0.166508,0.010035,0.018053,-0.268545,-1.090114,0.455020,0.757639,1.987062,0.999285,0.692423,...,-0.430248,-0.416324,-0.096056,-0.372181,-0.508712,-0.493789,-0.574343,-0.578735,1.736502,-0.580453
8061,0.202509,-0.017317,-0.267921,-0.589426,0.478543,-0.449163,-0.196672,-0.763750,-0.557585,-0.719922,...,-0.238893,-0.510057,-0.441859,-0.499372,-0.511246,-0.538075,-0.574343,1.727908,-0.575870,-0.580453


In [26]:
y_train

8701     1.0
4335     1.0
11203    1.0
11448    0.0
374      2.0
        ... 
9785     1.0
7763     2.0
5218     1.0
1346     1.0
3582     2.0
Name: Class, Length: 8063, dtype: float64

In [27]:
y_train.value_counts()

2.0    2716
0.0    2698
1.0    2649
Name: Class, dtype: int64

In [28]:
class_model = build_model(num_classes=3)

class_history = class_model.fit(
    X_train,
    y_train,
    validation_split=0.2,
    batch_size=32,
    epochs=50,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=3,
            restore_best_weights=True
        )
    ]
)

Epoch 1/50
202/202 [==============================] - 1s 3ms/step - loss: 1.0736 - accuracy: 0.4183 - val_loss: 1.0806 - val_accuracy: 0.4117
Epoch 2/50
202/202 [==============================] - 0s 2ms/step - loss: 0.9912 - accuracy: 0.4980 - val_loss: 1.0021 - val_accuracy: 0.4830
Epoch 3/50
202/202 [==============================] - 0s 2ms/step - loss: 0.9269 - accuracy: 0.5474 - val_loss: 0.9904 - val_accuracy: 0.5146
Epoch 4/50
202/202 [==============================] - 0s 2ms/step - loss: 0.8835 - accuracy: 0.5828 - val_loss: 0.9243 - val_accuracy: 0.5356
Epoch 5/50
202/202 [==============================] - 0s 2ms/step - loss: 0.8391 - accuracy: 0.6081 - val_loss: 0.9157 - val_accuracy: 0.5437
Epoch 6/50
202/202 [==============================] - 0s 2ms/step - loss: 0.7851 - accuracy: 0.6375 - val_loss: 0.8733 - val_accuracy: 0.5784
Epoch 7/50
202/202 [==============================] - 0s 2ms/step - loss: 0.7435 - accuracy: 0.6712 - val_loss: 0.8987 - val_accuracy: 0.5617
Epoch 

In [29]:
class_acc = class_model.evaluate(X_test, y_test, verbose=0)[1]
print("Test Accuracy (Class Model): {:.2f}%".format(class_acc * 100))

Test Accuracy (Class Model): 63.61%


### Predicting User

In [30]:
X_train, X_test, y_train, y_test = preprocess_inputs(data, target='User')

In [31]:
X_train

,AF3 delta std,AF3 delta m,AF3 theta std,AF3 theta m,AF3 alpha std,AF3 alpha m,AF3 beta std,AF3 beta m,F7 delta std,F7 delta m,...,AF4 delta m,AF4 theta std,AF4 theta m,AF4 alpha std,AF4 alpha m,AF4 beta std,AF4 beta m,Class_0.0,Class_1.0,Class_2.0
0,-0.592492,-0.628348,-0.961159,0.055905,0.463966,0.243742,-1.048130,-0.492935,-1.039009,-0.724031,...,-0.535664,-0.436216,-0.408949,-0.407029,-0.490410,-0.536367,-0.539566,-0.709147,1.429612,-0.712705
1,-1.215542,-1.060438,-0.687275,0.911497,0.266495,-0.045726,-0.996263,-1.073110,-1.595935,-1.789505,...,3.858153,2.577270,2.173543,3.159511,3.022871,0.657025,2.073117,-0.709147,1.429612,-0.712705
2,0.217034,0.061461,-0.302453,-0.276674,-0.609120,-0.737363,-1.144599,-1.598413,-1.092270,-1.223951,...,-0.537446,-0.400908,-0.385079,-0.267008,-0.348359,-0.541364,-0.569416,-0.709147,1.429612,-0.712705
3,2.636827,3.089887,0.715433,1.688836,1.115520,0.633049,-0.753087,-0.640998,0.643631,0.970393,...,-0.382336,0.141047,-0.008948,-0.427868,-0.301202,-0.505539,-0.522675,1.410145,-0.699491,-0.712705
4,-0.012351,0.039718,-0.098249,-0.213172,0.948769,-0.058219,-0.254097,-0.487547,0.332959,0.028684,...,-0.465464,-0.565201,-0.479292,-0.372068,-0.497185,-0.523806,-0.525149,-0.709147,-0.699491,1.403105
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8058,0.618517,1.150023,-0.957440,-0.735247,0.634060,1.144167,1.837545,2.507360,1.076999,1.533723,...,1.371221,-0.278668,0.153400,-0.380571,-0.152910,-0.500346,-0.453114,-0.709147,1.429612,-0.712705
8059,-0.157839,-0.326705,-0.099050,0.460753,-0.022372,-0.179817,1.098608,0.432161,-0.522874,-0.840956,...,1.627338,2.779410,1.612038,1.694883,4.498588,3.159293,2.567176,-0.709147,-0.699491,1.403105
8060,0.166508,0.010035,0.018053,-0.268545,-1.090114,0.455020,0.757639,1.987062,0.999285,0.692423,...,-0.167387,-0.430248,-0.416324,-0.096056,-0.372181,-0.508712,-0.493789,-0.709147,1.429612,-0.712705
8061,0.202509,-0.017317,-0.267921,-0.589426,0.478543,-0.449163,-0.196672,-0.763750,-0.557585,-0.719922,...,-0.463766,-0.238893,-0.510057,-0.441859,-0.499372,-0.511246,-0.538075,-0.709147,1.429612,-0.712705


In [32]:
y_train

8701     3
4335     0
11203    3
11448    3
374      1
        ..
9785     2
7763     0
5218     2
1346     1
3582     2
Name: User, Length: 8063, dtype: int64

In [33]:
y_train.value_counts()

3    2032
1    2023
2    2008
0    2000
Name: User, dtype: int64

In [47]:
user_model = build_model(num_classes=4)

user_history = user_model.fit(
    X_train,
    y_train,
    validation_split=0.2,
    batch_size=32,
    epochs=50,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=3,
            restore_best_weights=True
        )
    ]
)

Epoch 1/50
202/202 [==============================] - 2s 4ms/step - loss: 0.1278 - accuracy: 0.9614 - val_loss: 0.0246 - val_accuracy: 0.9938
Epoch 2/50
202/202 [==============================] - 0s 2ms/step - loss: 0.0162 - accuracy: 0.9955 - val_loss: 0.0137 - val_accuracy: 0.9957
Epoch 3/50
202/202 [==============================] - 0s 2ms/step - loss: 0.0086 - accuracy: 0.9972 - val_loss: 0.0067 - val_accuracy: 0.9981
Epoch 4/50
202/202 [==============================] - 0s 2ms/step - loss: 0.0033 - accuracy: 0.9995 - val_loss: 0.0051 - val_accuracy: 0.9988
Epoch 5/50
202/202 [==============================] - 0s 2ms/step - loss: 0.0037 - accuracy: 0.9989 - val_loss: 0.0059 - val_accuracy: 0.9981
Epoch 6/50
202/202 [==============================] - 0s 2ms/step - loss: 0.0019 - accuracy: 0.9995 - val_loss: 0.0063 - val_accuracy: 0.9969
Epoch 7/50
202/202 [==============================] - 0s 2ms/step - loss: 5.1754e-04 - accuracy: 1.0000 - val_loss: 0.0039 - val_accuracy: 0.9981
Ep

In [37]:
user_acc = user_model.evaluate(X_test, y_test, verbose=0)[1]
print("Test Accuracy (User Model): {:.2f}%".format(user_acc * 100))

Test Accuracy (User Model): 99.94%


**Conclusion** :After doing both the predictions we can conclude that it's easier for machine learning model to predict the user based on the data than it is to predict the hand movement.